In [30]:
import numpy as np
import gymnasium as gym
from gymnasium import spaces
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import time
import random
import pygame
from loguru import logger

# --- Constants ---
SCREEN_WIDTH = 1200
SCREEN_HEIGHT = 800
FPS = 60

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (220, 50, 50)
GREEN = (50, 220, 50)
BLUE = (50, 50, 220)
GRAY = (150, 150, 150)

In [105]:
import gymnasium as gym
from gymnasium.spaces import Box, Discrete,Tuple
import numpy as np
import pygame

# Define colors
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)  # Color for the trajectory

class CustomEnv(gym.Env):
    metadata = {"render_modes": ["human", "rgb_array"], "render_fps": 60}

    def __init__(self):
        super().__init__()
        self.grid_size = 10
        self.obstacle_num = 8
        self.distance_threshold_goal = 0.3  # Tolerance for goal proximity

        # Define two separate thresholds for obstacle handling
        self.distance_threshold_penalty = 1  # Penalty zone threshold (larger value)
        self.distance_threshold_collision = 0.5  # Collision threshold (smaller value)
        self.penalty_factor = 25  # Penalty scaling factor

        self.steps = 0

        self.pre_distance = 0  # Previous distance to goal, used for reward shaping
        self.current_distance = 0  # Current distance to goal, used for reward shaping
        # Action space (dx, dy)
        self.action_space = spaces.Box(low=-1, high=1, shape=(2,), dtype=np.float32)
        # Observation space (robot_x, robot_y, goal_x, goal_y)
        self.observation_shape = 2 + 2 + 1+self.obstacle_num * 2  # Robot position, goal position, and obstacle positions
        self.observation_space = Box(low=0, high=self.grid_size, shape=(self.observation_shape,), dtype=np.float32)

        
        # For rendering
        self.window = None
        self.clock = None
        self.cell_size = 50 # Pixels per grid unit
        self.trajectory_points = [] # New: List to store past robot positions

    def _get_obs(self):
   
        return np.concatenate(([self.robot_position]+ [self.goal_position]+[np.array([self.closest_obstacle_distance])]+self.obstacle_position),dtype=np.float32)

    def _get_info(self):
        return {
            "distance_to_goal": np.linalg.norm(self.robot_position - self.goal_position)
        }

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.robot_position = np.random.uniform(0, self.grid_size, size=2)
        self.goal_position = np.random.uniform(0, self.grid_size, size=2)
        self.obstacle_position = [np.random.uniform(0, self.grid_size, size=2) for _ in range(self.obstacle_num)]
        self.closest_obstacle_distance = np.min([np.linalg.norm(self.robot_position - obs) for obs in self.obstacle_position])
        self.steps = 0
        self.trajectory_points = [self.robot_position.copy()] # New: Reset trajectory and add initial position
        self.pre_distance = np.linalg.norm(self.robot_position - self.goal_position)
        self.max_steps = 1000  # Set a maximum number of steps to prevent infinite loops
        return self._get_obs(), self._get_info()

    def _reward(self):
        terminated = False
        truncated = False
        reward = 0  # Initialize reward
        done_reason = None  # Initialize done reason
        if self.steps >= self.max_steps:
            reward -= 50
            truncated = True  # Truncate if max steps reached
        
        

        if np.any(self.robot_position <= 0.0) or np.any(self.grid_size-self.robot_position <=0.0):
            reward -= 100
            terminated = True
            done_reason = "Out of bounds"

        self.robot_position = np.clip(self.robot_position, 0, self.grid_size)

        # Auxiliary Rewards -  distance to goal
        self.current_distance = np.linalg.norm(self.robot_position - self.goal_position)
        reward += (self.pre_distance - self.current_distance)*40  # Reward shaping based on distance change
        self.pre_distance = self.current_distance

        # Obstacle handling with two thresholds
        self.closest_obstacle_distance = np.min([np.linalg.norm(self.robot_position - obs) for obs in self.obstacle_position])
        for obs in self.obstacle_position:
            distance = np.linalg.norm(self.robot_position - obs)
            if distance < self.distance_threshold_collision:
                # Game over - collision detected
                reward -= 150  # Large negative reward for collision
                terminated = True
                done_reason = "Collision with obstacle"
                break
            elif distance < self.distance_threshold_penalty:
                reward -= (self.distance_threshold_penalty - distance)* 200  # 或者更大
        
             
        
        if np.linalg.norm(self.robot_position - self.goal_position) < self.distance_threshold_goal:
            reward += 100
            terminated = True
            done_reason = "Reached the goal"
        
        reward-=  0.5  # Small penaty for  each step to encourage efficiency

        return reward,terminated,truncated,done_reason
        

    def step(self, action):
        stride = 0.3  # Base stride length
        self.robot_position += action * stride  # Scale the action to control speed

        self.trajectory_points.append(self.robot_position.copy()) # New: Add current position to trajectory

        reward,terminated,truncated,done_reason = self._reward()

        info = self._get_info()
        observation = self._get_obs()
        self.steps += 0.01

        # if terminated or truncated:
        #     logger.warning(f"Episode ended: {done_reason}, Steps: {self.steps:.2f}")
        return observation, reward, terminated, truncated, info

    def render(self):
        if self.window is None:
            pygame.init()
            pygame.display.init()
            self.window = pygame.display.set_mode(
                (int(self.grid_size * self.cell_size), int(self.grid_size * self.cell_size))
            )
            pygame.display.set_caption("CustomEnv")
        if self.clock is None:
            self.clock = pygame.time.Clock()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                import sys
                sys.exit() # Exit the program
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 1: # Left mouse button click
                    mouse_x, mouse_y = event.pos # Get mouse position in pixels
                    # Convert pixel coordinates to grid coordinates
                    new_goal_x = mouse_x / self.cell_size
                    new_goal_y = mouse_y / self.cell_size
                    self.goal_position = np.array([new_goal_x, new_goal_y], dtype=np.float32)
                    print(f"New goal set at grid position: {self.goal_position}")


        canvas = pygame.Surface((self.grid_size * self.cell_size, self.grid_size * self.cell_size))
        canvas.fill(WHITE)
        virus_image = pygame.image.load("病毒.png").convert_alpha()  # Load an image if needed, but not used here
        robot_image = pygame.transform.scale(virus_image, (int(self.cell_size * 0.8), int(self.cell_size * 0.8)))  # Scale the image
        # New: Draw the trajectory
        if len(self.trajectory_points) > 1:
            scaled_points = []
            for point in self.trajectory_points:
                scaled_points.append((int(point[0] * self.cell_size), int(point[1] * self.cell_size)))
            
            # Draw lines between consecutive points
            pygame.draw.lines(canvas, BLUE, False, scaled_points, 2) # Blue line, not closed, 2 pixels wide
            
            # Optionally, draw small circles at each point to emphasize
            for point_coord in scaled_points:
                pygame.draw.circle(canvas, BLUE, point_coord, 3) # Small blue circles

        # Draw robot
        pygame.draw.circle(
            canvas,
            RED,
            (int(self.robot_position[0] * self.cell_size), int(self.robot_position[1] * self.cell_size)),
            int(self.cell_size * 0.2)
        )
        # Draw obstacles
        for obs in self.obstacle_position:
            # pygame.draw.circle(
            #     canvas,
            #     GRAY,
            #     (int(obs[0] * self.cell_size), int(obs[1] * self.cell_size)),
            #     int(self.cell_size * 0.5)
            # )
            canvas.blit(robot_image, (int(obs[0] * self.cell_size), int(obs[1] * self.cell_size)))
        # Draw goal
        pygame.draw.circle(
            canvas,
            GREEN,
            (int(self.goal_position[0] * self.cell_size), int(self.goal_position[1] * self.cell_size)),
            int(self.cell_size * 0.3)
        )

        self.window.blit(canvas, canvas.get_rect())
        pygame.event.pump()
        pygame.display.flip()
        self.clock.tick(self.metadata["render_fps"])

    def close(self):
        if self.window is not None:
            pygame.display.quit()
            pygame.quit()





In [106]:
import gymnasium as gym
import torch
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.monitor import Monitor


env =DummyVecEnv([lambda: Monitor(CustomEnv())])  # Wrap the environment with Monitor for logging
policy_kwargs = dict(
    net_arch=[dict(pi=[64,128], vf=[128,128])],
    # activation_fn=torch.nn.ReLU  # 改为 ReLU，通常更适合稀疏奖励
)

model = PPO("MlpPolicy", env, verbose=1, ent_coef=0.005,policy_kwargs=policy_kwargs,tensorboard_log="./ppo_custom_env_tensorboard/")
# model.load("logs/best_model/best_model.zip",env=env)  # Load the best model
eval_callback = EvalCallback(
    env,
    best_model_save_path='./logs/best_model2/',
    log_path = './logs/',
    eval_freq=100000,  # 每1000步评估一次
    deterministic=True,
    render=False,
    n_eval_episodes=5,  # 每次评估5个episode
)

model.learn(total_timesteps=5000000,callback=eval_callback)
# env.save("vec_env.pkl")  # Save the environment state
# vec_env = model.get_env()
# obs = vec_env.reset()
# for i in range(1000):
#     action, _states = model.predict(obs, deterministic=True)
#     obs, reward, dones, info = vec_env.step(action)
    # vec_env.render()


env.close()

Using cpu device
Logging to ./ppo_custom_env_tensorboard/PPO_36


g:\anaconda\envs\RL\lib\site-packages\stable_baselines3\common\policies.py:486: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 120      |
|    ep_rew_mean     | -286     |
| time/              |          |
|    fps             | 2801     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 90.6       |
|    ep_rew_mean          | -281       |
| time/                   |            |
|    fps                  | 1919       |
|    iterations           | 2          |
|    time_elapsed         | 2          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.01353144 |
|    clip_fraction        | 0.148      |
|    clip_range           | 0.2        |
|    entropy_loss         | -2.83      |
|    explained_variance   | 0.00111    |
|    learning_rate        | 0.0003     |
|   

KeyboardInterrupt: 

In [101]:
env = CustomEnv()
model.load("logs/best_model/best_model.zip")  # Load the best model
obs,_ = env.reset()
print(obs)
for i in range(1000000):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, teminated,_, info = env.step(action)
    print(action,info["distance_to_goal"])
    print(obs)
    print(action)
    print("Reward:", reward)
    print(env.robot_position)
    env.render()
    time.sleep(0.6)  # Control the frame rate
    if teminated:
        env.close()
        break
        print("Resetting environment")



[9.0852375  9.894207   8.377251   4.4080257  0.53639656 0.8593423
 7.7813177  9.61936    9.844867   1.698531   0.30505937 0.40137142
 1.4154787  3.7049382  3.8755114  7.5925903  2.2259498  8.708509
 7.7482805  6.1647043  4.84906    0.        ]
[-0.18751201 -0.15672167] 5.478071359209566
[9.028984   9.84719    8.377251   4.4080257  0.59038067 0.8593423
 7.7813177  9.61936    9.844867   1.698531   0.30505937 0.40137142
 1.4154787  3.7049382  3.8755114  7.5925903  2.2259498  8.708509
 7.7482805  6.1647043  4.84906    0.        ]
[-0.18751201 -0.15672167]
Reward: -80.27972545117694
[9.02898395 9.84719002]
[-0.18692452 -0.15491629] 5.425486731162481
[8.972907   9.800715   8.377251   4.4080257  0.6479595  0.8593423
 7.7813177  9.61936    9.844867   1.698531   0.30505937 0.40137142
 1.4154787  3.7049382  3.8755114  7.5925903  2.2259498  8.708509
 7.7482805  6.1647043  4.84906    0.01      ]
[-0.18692452 -0.15491629]
Reward: -68.8047253062037
[8.97290659 9.80071514]
[-0.18639484 -0.1530364 ] 5

SystemExit: 

g:\anaconda\envs\RL\lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [11]:
import cv2
import numpy as np
import mediapipe as mp
from segment_anything import sam_model_registry, SamPredictor


img = cv2.imread('../test.jpg')
sam = sam_model_registry["vit_b"](checkpoint="../sam_vit_b_01ec64.pth")
predictor = SamPredictor(sam)
predictor.set_image(img)


mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

hands = mp_hands.Hands(static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
results = hands.process(rgb)
if results.multi_hand_landmarks:
    hand_landmarks = results.multi_hand_landmarks[0]
    x = hand_landmarks.landmark[0].x
    y =hand_landmarks.landmark[0].y
point_coords = np.array([[x, y]])
print(point_coords.shape)
point_labels = np.array([1])   # 1=前景, 0=背景

masks, scores, logits = predictor.predict(
    point_coords=point_coords,
    point_labels=point_labels,
    multimask_output=True,  # 返回多候选，配合 scores/IoU 选最优
)
best = masks[scores.argmax()]  # 挑最优掩膜



hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

lower = np.array([0, 30, 60], dtype="uint8")
upper = np.array([20, 150, 255], dtype="uint8")
mask = cv2.inRange(hsv, lower, upper)

cv2.imshow("mask", best)
cv2.waitKey(0)
cv2.destroyAllWindows()

g:\anaconda\envs\RL\lib\site-packages\segment_anything\build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


(1, 2)


error: OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'imshow'
> Overload resolution failed:
>  - mat data type = bool is not supported
>  - Expected Ptr<cv::cuda::GpuMat> for argument 'mat'
>  - Expected Ptr<cv::UMat> for argument 'mat'


In [2]:
import json

with open(r'C:\Users\admin\Desktop\科研\RL\src\logs\best_model_sac14\env_args.json', 'r') as f:
    data = json.load(f)
    print(data)

{'grid_size': 10, 'distance_threshold_penalty': 4, 'distance_threshold_collision': 1.5, 'penalty_factor': 5, 'distance_reward_factor': 5, 'smooth_action_penalty': 1, 'max_steps': 40, 'margin': 0.1, 'reward_step': 4, 'reward_max_step': 50, 'reward_bound': -400, 'reward_arm': -200, 'reward_hand': -200}
